# A/B Testing Analysis

In [2]:
import numpy as np
import pandas as pd

from scipy import stats
import seaborn as sns

# Objectives

- Conduct an A/B test in Python
- Interpret the results of the A/B tests for a stakeholder

# Example Together

## Question

We have data about whether customers completed sales transactions, segregated by the type of ad banners to which the customers were exposed.

The question we want to answer is whether there was any difference in sales "conversions" between desktop customers who saw the sneakers banner and desktop customers who saw the accessories banner in the month of May 2019.

## Considerations

What would we need to consider when designing our experiment?

Might include:

- Who is it that we're including in our test?
- How big of an effect would make it "worth" us seeing?
    - This can affect sample size
    - This can give context of a statistically significant result
- Other biases or "gotchas"

## Loading the Data

First let's download the data from [kaggle](https://www.kaggle.com/podsyp/how-to-do-product-analytics) via the release page of this repo: https://github.com/flatiron-school/ds-ab_testing/releases 

The code below will load it into our DataFrame:

In [3]:
# This will download the data from online so it can take
# some time (but relatively small download)

df = pd.read_csv('https://github.com/flatiron-school/ds-ab_testing/releases/download/v1.2/products_small.csv')

> Let's take a look while we're at it

In [4]:
df.head()

,Unnamed: 0,order_id,user_id,page_id,product,site_version,time,title,target
0,4122928,3e6c5e89fdddcaee0eed210ec2c9cadf,90d58d967eb72656e86059ec6f208092,2fdc16a09e0016555dd4da4a3fe84414,accessories,desktop,2019-03-06 08:42:47,banner_show,0
1,564306,feed6203517d3abf6aab13761633174b,08703dab1f004eabba25aacb7f0e5484,6b0a902b9b73d5a158d0119d6feb38ac,sneakers,mobile,2019-04-19 18:30:45,banner_show,0
2,1872289,e33d5d7941edc281646aa37763729771,bdf1d25697e21419901c94fabdafad15,9ddb7315c4357929931b48f2b3d11c62,company,mobile,2019-01-20 17:20:10,banner_show,0
3,3616779,7c4caa8d508fa7c3bbc25f35cdd9168a,8d2f23a732c9527d95678088a3bac122,1f86cd0bea31d54a5b511b42fd19401a,sneakers,mobile,2019-02-20 09:38:32,banner_show,0
4,5871482,12874b29bde8bbd43fb2b95735caf9e6,5a22604f8f31ae98ee1211ece3a02004,b533f7e2003418c63fd71471264c559a,sneakers,mobile,2019-04-24 09:19:02,banner_show,0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 9 columns):
 #   Column        Non-Null Count    Dtype 
---  ------        --------------    ----- 
 0   Unnamed: 0    1000000 non-null  int64 
 1   order_id      1000000 non-null  object
 2   user_id       1000000 non-null  object
 3   page_id       1000000 non-null  object
 4   product       1000000 non-null  object
 5   site_version  1000000 non-null  object
 6   time          1000000 non-null  object
 7   title         1000000 non-null  object
 8   target        1000000 non-null  int64 
dtypes: int64(2), object(7)
memory usage: 68.7+ MB


In [6]:
df.describe(include=['object'])

,order_id,user_id,page_id,product,site_version,time,title
count,1000000,1000000,1000000,1000000,1000000,1000000,1000000
unique,1000000,882589,959023,5,2,952092,3
top,04a01ecbc1179dde79d3186efc4f9401,2a98f631f947487fc710b89d13d5a51a,a3d2de7675556553a5f08e4c88d2c228,clothes,mobile,2019-01-14 18:47:47,banner_show
freq,1,8,29395,210996,718521,5,872275


## Some Exploration to Better Understand our Data

Lets's look at the different banner types:

In [7]:
df['product'].value_counts()

clothes             210996
company             203020
sneakers            201298
sports_nutrition    193200
accessories         191486
Name: product, dtype: int64

In [8]:
df.groupby('product')['target'].value_counts()

product           target
accessories       0         186127
                  1           5359
clothes           0         197720
                  1          13276
company           0         203020
sneakers          0         193375
                  1           7923
sports_nutrition  0         190363
                  1           2837
Name: target, dtype: int64

In [9]:
df.groupby('product').get_group('accessories')

,Unnamed: 0,order_id,user_id,page_id,product,site_version,time,title,target
0,4122928,3e6c5e89fdddcaee0eed210ec2c9cadf,90d58d967eb72656e86059ec6f208092,2fdc16a09e0016555dd4da4a3fe84414,accessories,desktop,2019-03-06 08:42:47,banner_show,0
9,2906884,322c56563d9aa57f85e38613cf16775c,c0eb4010a42b5d2805097075262b5e7f,88aa92f29098b7c6c6cab6069f1d136b,accessories,mobile,2019-01-30 11:33:16,banner_show,0
10,3454132,9eb48d87797e94aca6ee100103518baf,41236cf8d07200ed4bc945a7a4eca453,92a5355bb9f84f557888a8727b901f1b,accessories,mobile,2019-03-13 17:53:30,banner_show,0
13,2213782,d8b72d7a4907f06cea99f36e71c20b2e,5291a099f39803cecb6851a07942a397,822551f4faa31d922c2c61df438c9623,accessories,mobile,2019-01-18 16:49:20,banner_show,0
14,7002267,817c9b4833b5212ec823352e5fefb5f4,bbac285a887bd22d70c10f4ea9671d8a,1d9d9556639b616a3cad9672969b3f39,accessories,mobile,2019-03-03 15:14:33,banner_show,0
...,...,...,...,...,...,...,...,...,...
999976,5332512,43ddc2e418dca4c80305be5f4eeb8880,4cf095b559f7f162cd8e59795d0117b5,cc763b5231042584b4b1497623b51e7a,accessories,mobile,2019-03-25 10:39:07,banner_show,0
999977,608046,1f64ea586b6f72d4526fa5ed00951041,2ecf5217d3dd802d5259a54e708f438a,86bf6039dbce651e3dd05a4833d649fd,accessories,desktop,2019-02-24 21:46:36,banner_show,0
999981,2061651,de970a570172173a5f0c4bd898439f24,3779a65742b97ea0615d962a0b6a7275,a980dbe8ca84017ed20dcfdac19eba52,accessories,mobile,2019-04-01 18:13:17,banner_show,0
999982,4056250,6472ffae8c229b7ac388f2599093068d,3719d2c9ad4df119e1004322f5740045,2865e0949360179dc3d7dfe4e1aebcf2,accessories,desktop,2019-02-26 17:14:24,banner_show,0


In [10]:
df.groupby(['product', 'target']).count()

Unnamed: 0  order_id  user_id  page_id  site_version  \
product          target                                                         
accessories      0           186127    186127   186127   186127        186127   
                 1             5359      5359     5359     5359          5359   
clothes          0           197720    197720   197720   197720        197720   
                 1            13276     13276    13276    13276         13276   
company          0           203020    203020   203020   203020        203020   
sneakers         0           193375    193375   193375   193375        193375   
                 1             7923      7923     7923     7923          7923   
sports_nutrition 0           190363    190363   190363   190363        190363   
                 1             2837      2837     2837     2837          2837   

                           time   title  
product          target                  
accessories      0       186127  186127  
                 1         5359    5359  
clothes          0       197720  197720  
                 1        13276   13276  
company          0       203020  203020  
sneakers         0       193375  193375  
                 1         7923    7923  
sports_nutrition 0       190363  190363  
                 1         2837    2837

In [11]:
df.groupby(['product', 'target']).agg('count')

Unnamed: 0  order_id  user_id  page_id  site_version  \
product          target                                                         
accessories      0           186127    186127   186127   186127        186127   
                 1             5359      5359     5359     5359          5359   
clothes          0           197720    197720   197720   197720        197720   
                 1            13276     13276    13276    13276         13276   
company          0           203020    203020   203020   203020        203020   
sneakers         0           193375    193375   193375   193375        193375   
                 1             7923      7923     7923     7923          7923   
sports_nutrition 0           190363    190363   190363   190363        190363   
                 1             2837      2837     2837     2837          2837   

                           time   title  
product          target                  
accessories      0       186127  186127  
                 1         5359    5359  
clothes          0       197720  197720  
                 1        13276   13276  
company          0       203020  203020  
sneakers         0       193375  193375  
                 1         7923    7923  
sports_nutrition 0       190363  190363  
                 1         2837    2837

Let's look at the range of time-stamps on these data:

In [12]:
df['time'].min()

'2019-01-01 00:00:25'

In [13]:
df['time'].max()

'2019-05-31 23:59:21'

Let's check the counts of the different site_version values:

In [15]:
df['site_version'].value_counts()

mobile     718521
desktop    281479
Name: site_version, dtype: int64

And now check titles

In [18]:
df['title'].value_counts()

banner_show     872275
banner_click     98330
order            29395
Name: title, dtype: int64

In [19]:
len(df.loc[df['title'] == 'order'])

29395

In [20]:
sum(df['target'])

29395

In [21]:
df.groupby('title').agg({'target': 'mean'})

,target
title,
banner_click,0
banner_show,0
order,1


## Experimental Setup

We need to filter by site_version, time, and product:

In [22]:
# Time
df.loc[df['time'] >= '2019-05-01']

,Unnamed: 0,order_id,user_id,page_id,product,site_version,time,title,target
19,4880034,c59e23bca9e3e74b76e5c89591920741,885540374a1549d83561b2a40313ff2e,503567af06c98967c3cc1b97886f9a36,clothes,desktop,2019-05-15 13:40:11,banner_show,0
21,5368605,544a8db198f2c93f9efe020d0ccacb47,6882b941b57ca76302c9b7bceac19586,7f6a5bebd21c5bb052219ea59c756435,company,mobile,2019-05-23 20:33:13,banner_click,0
27,7899150,af5361b16580e2589e80d7c5e3ed0a3c,afea2091e55fc90f8b09923a2a71705f,40d319b24b329442675ea5c06f53dd29,sneakers,mobile,2019-05-21 16:46:47,banner_show,0
32,8064199,f909eb9bbf2795337dd38323b7951705,5a96585c3db94eb2ae40e559cbd2e164,94b6ea2316b773afa20925d9b79d5bf8,accessories,desktop,2019-05-19 09:26:29,banner_show,0
33,8394794,0287fbf70d2d13fb50a082af56925baa,d3921696f5f6d83a4fa112dde7b3b380,6338719e3f54583dff8d010e6673776e,sneakers,mobile,2019-05-28 19:30:57,banner_show,0
...,...,...,...,...,...,...,...,...,...
999980,8159492,f49debd6ec37e7dc963cc8f9263a040f,10167ef9d0127d225909c4ece2cdfc28,6b668b223896c8683f14d20d332b607e,sports_nutrition,mobile,2019-05-21 16:50:47,banner_show,0
999985,8197096,9b90203a458a19bec09cd4fb6f9dc69f,bff8aee79342f86d1346d94651e84a64,876a5dd1c118fa00690f852d9ed6ad6d,company,desktop,2019-05-22 06:30:48,banner_show,0
999992,7735711,c40020044e86083b53c48ec23b1eda2a,fbb6f9203a0845ab8a824855582059c3,eeae53a3fe2725c7e1d57abed2b116cb,clothes,desktop,2019-05-02 16:00:12,banner_show,0
999994,6401235,f49e74785bb20dbc3baa646edf13bfc5,e43bfac12750d537c89e9332c376279a,f67fabd6264199d7f0bd7e24e8c0dabc,sports_nutrition,mobile,2019-05-16 17:15:10,banner_show,0


In [26]:
# All
df_AB = df.loc[(df['site_version'] == 'desktop') & (df['time'] >= '2019-05-01') & 
              ((df['product'] == 'accessories') | (df['product'] == 'sneakers'))]

In [35]:
banner_list = ['accessories', 'sneakers']

In [37]:
df_AB_3 = df.loc[(df['site_version'] == 'desktop') & (df['time'] >= '2019-05-01') &
                (df['product'].isin(banner_list))]

In [27]:
# Not going to work without nested OR conditional
df_AB_2 = df.loc[(df['site_version'] == 'desktop') & (df['time'] >= '2019-05-01') &
              (df['product'] == 'accessories') | (df['product'] == 'sneakers')]

In [28]:
df_AB.tail()

,Unnamed: 0,order_id,user_id,page_id,product,site_version,time,title,target
999904,5738290,ecfe9da2dbea255f05b2f526a4c663a7,cd7598ba5798ad6bd3f5a667ae4c217a,644ad50dd900d6735d8ceb15b71d50f8,accessories,desktop,2019-05-01 23:01:05,banner_show,0
999908,7885931,8c28b45ff7fbb5d5df0b7670183689db,2f28106a4db2babcfde1c4095c805632,a3d2de7675556553a5f08e4c88d2c228,sneakers,desktop,2019-05-26 11:19:09,order,1
999915,8099155,c613db2f19c7285f101196062c178490,97473368941b89c4716c4f389e9b8bd7,a3d2de7675556553a5f08e4c88d2c228,sneakers,desktop,2019-05-11 09:04:46,order,1
999930,6690689,6f7f32ecf6790e4eb06384b479f43162,73f403b4843c70c41c0db294ee479582,13f395d7c86ef276f7ce52a557fd3491,accessories,desktop,2019-05-09 09:17:24,banner_show,0
999999,7650805,af3a19fcceb477d2992f1771af3a9b34,515cb969971fb5e08ae0316f58e3f434,ef38d906f15db1efde867c68f1336946,sneakers,desktop,2019-05-20 17:58:16,banner_show,0


In [29]:
df_AB_2.shape

(214001, 9)

In [34]:
df_AB_2['site_version'].value_counts()

mobile     146261
desktop     67740
Name: site_version, dtype: int64

In [31]:
df_AB.shape

(25972, 9)

In [38]:
df_AB_3.shape

(25972, 9)

In [32]:
df_AB['product'].value_counts()

sneakers       13269
accessories    12703
Name: product, dtype: int64

In [33]:
df_AB['site_version'].value_counts()

desktop    25972
Name: site_version, dtype: int64

### What Test Would Make Sense?

Since we're comparing the frequency of conversions of customers who saw the "sneakers" banner against those who saw the "accessories" banner, we can use a $\chi^2$ test.

Note there are other hypothesis tests we can use but this should be fine since it should fit our criteria.

### The Hypotheses

$H_0$: Customers who saw the sneakers banner were no more or less likely to buy than customers who saw the accessories banner.

$H_1$: Customers who saw the sneakers banner were more or less likely to buy than customers who saw the accessories banner.

### Setting a Threshold

We'll set a false-positive rate of $\alpha = 0.05$.

## $\chi^2$ Test

### Setup the Data

We need our contingency table: the numbers of people who did or did not submit orders, both for the accessories banner and the sneakers banner. 

In [39]:
counts = df_AB.groupby(['product', 'target']).count()['title']
counts

product      target
accessories  0         12207
             1           496
sneakers     0         12470
             1           799
Name: title, dtype: int64

In [40]:
df_A = df_AB.loc[df_AB['product'] == 'accessories']
df_B = df_AB.loc[df_AB['product'] == 'sneakers']

In [41]:
accessories_orders = sum(df_A['target'])
sneakers_orders = sum(df_B['target'])

accessories_orders, sneakers_orders

(496, 799)

In [42]:
accessories_orders = counts[1]
accessories_orders

496

In [43]:
df_AB['product'].value_counts()

sneakers       13269
accessories    12703
Name: product, dtype: int64

To get the numbers of people who didn't submit orders, we get the total number of people who were shown banners and then subtract the numbers of people who did make orders.

In [44]:
accessories_total = len(df_A)
sneakers_total = len(df_B)

accessories_no_orders = accessories_total - accessories_orders
sneakers_no_orders = sneakers_total - sneakers_orders

accessories_no_orders, sneakers_no_orders

(12207, 12470)

In [45]:
contingency_table = np.array([[accessories_orders, accessories_no_orders],
                              [sneakers_orders, sneakers_no_orders]])

contingency_table

array([[  496, 12207],
       [  799, 12470]])

In [46]:
contin_list = list([[accessories_orders, accessories_no_orders], 
                   [sneakers_orders, sneakers_no_orders]])

contin_list

[[496, 12207], [799, 12470]]

In [48]:
pd.DataFrame([[accessories_orders, accessories_no_orders], 
                   [sneakers_orders, sneakers_no_orders]], columns=['order', 'no_order'])

,order,no_order
0,496,12207
1,799,12470


### Calculation


In [49]:
stats.chi2_contingency(contingency_table)

(60.94640624666488,
 5.86498985882812e-15,
 1,
 array([[  633.38922686, 12069.61077314],
        [  661.61077314, 12607.38922686]]))

In [50]:
stats.chi2_contingency(pd.DataFrame([[accessories_orders, accessories_no_orders], 
                   [sneakers_orders, sneakers_no_orders]]))

(60.94640624666488,
 5.86498985882812e-15,
 1,
 array([[  633.38922686, 12069.61077314],
        [  661.61077314, 12607.38922686]]))

In [54]:
stats, p, df, exp = stats.chi2_contingency(contin_list)

In [56]:
alpha = .05

In [57]:
p < alpha

True

This extremely low $p$-value suggests that these two groups are genuinely performing differently. In particular, the desktop customers who saw the sneakers banner in May 2019 bought at a higher rate than the desktop customers who saw the accessories banner in May 2019.

In [52]:
(sneakers_orders / sneakers_total) * 100


6.021553998040545

In [53]:
(accessories_orders / accessories_total) * 100

3.9045894670550267

## Interpretation

In [58]:
contingency_table

array([[  496, 12207],
       [  799, 12470]])

In [60]:
contingency_table[:, 0] 

array([496, 799])

In [61]:
contingency_table.sum(axis=1)

array([12703, 13269])

In [59]:
contingency_table[:, 0] / contingency_table.sum(axis=1)

array([0.03904589, 0.06021554])

In [62]:
# Find the difference in conversion rate
accessory_CR, sneaker_CR = contingency_table[:, 0] / contingency_table.sum(axis=1)

In [63]:
print(f'Conversion Rate for accessory banner:\n\t{100*accessory_CR:.3f}%')
print(f'Conversion Rate for sneaker banner:\n\t{100*sneaker_CR:.3f}%')
print('')
print(f'Absolute difference of CR: {100*(sneaker_CR-accessory_CR):.3f}%')

Conversion Rate for accessory banner:
	3.905%
Conversion Rate for sneaker banner:
	6.022%

Absolute difference of CR: 2.117%


In [64]:
accessories_total, sneakers_total

(12703, 13269)

In [66]:
496 / accessories_total * 100

3.9045894670550267

In [67]:
799 / sneakers_total * 100

6.021553998040545

So we can say:
- There was a statistically significant difference at the $\alpha$-level (confidence level)
- The difference was about $2\%$ in favor of the sneaker banner!

# Exercise

> The company is impressed with what you found and is now wondering if there is a difference in their other banner ads!

With your group, look at the same month (May 2019) but compare different platforms ('mobile' vs 'desktop') and or different banner types ('accessories', 'sneakers', 'clothes', 'sports_nutrition'). Just don't repeat the same test we did above 😉

Make sure you record what considerations you have for the experiment, what hypothesis test you performed ($H_0$ and $H_1$ too), and your overall conclusion/interpretation for the _business stakeholders_. Is there a follow-up you'd suggest? 